In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

Importando Bibliotecas necessárias

Agora,Importar CSV

In [2]:
base = pd.read_csv('https://raw.githubusercontent.com/Gustavolorenzz/IA/main/Bases/student-por.csv', sep=';')
colunas_desejadas = ['absences','failures', 'higher','G1','G2','G3']

base = base[colunas_desejadas]
base

,absences,failures,higher,G1,G2,G3
0,4,0,yes,0,11,11
1,2,0,yes,9,11,11
2,6,0,yes,12,13,12
3,0,0,yes,14,14,14
4,0,0,yes,11,13,13
...,...,...,...,...,...,...
644,4,1,yes,10,11,10
645,4,0,yes,15,15,16
646,6,0,yes,11,12,9
647,6,0,yes,10,10,10


Mapeamento de Qualificação

In [3]:
def mapear_qualificacao(nota):
    if nota >= 17.5:
        return 'Excellent'
    elif nota >= 15.5:
        return 'Very good'
    elif nota >= 13.5:
        return 'Good'
    elif nota >= 9.5:
        return 'Sufficient'
    elif nota >= 3.5:
        return 'Weak'
    else:
        return 'Poor'

# Aplicar a função às colunas G1, G2 e G3
base['G1'] = base['G1'].apply(mapear_qualificacao)
base['G2'] = base['G2'].apply(mapear_qualificacao)
base['G3'] = base['G3'].apply(mapear_qualificacao)

In [4]:
base

,absences,failures,higher,G1,G2,G3
0,4,0,yes,Poor,Sufficient,Sufficient
1,2,0,yes,Weak,Sufficient,Sufficient
2,6,0,yes,Sufficient,Sufficient,Sufficient
3,0,0,yes,Good,Good,Good
4,0,0,yes,Sufficient,Sufficient,Sufficient
...,...,...,...,...,...,...
644,4,1,yes,Sufficient,Sufficient,Sufficient
645,4,0,yes,Good,Good,Very good
646,6,0,yes,Sufficient,Sufficient,Weak
647,6,0,yes,Sufficient,Sufficient,Sufficient


In [5]:
def mapear_faltas(absences):
  if absences >= 10:
    return 'High level of absences'
  elif absences >= 5:
    return 'Medium level of absences'
  else:
    return 'Low level of absences'


#Aplicar função à coluna absences
base['absences'] = base['absences'].apply(mapear_faltas)

In [6]:
label_encoder_higher = LabelEncoder()
label_encoder_g1 = LabelEncoder()
label_encoder_g2 = LabelEncoder()
label_encoder_absences = LabelEncoder()

In [7]:
# Aplique o label encoder às colunas desejadas
base['higher'] = label_encoder_higher.fit_transform(base['higher'])
base['G1'] = label_encoder_g1.fit_transform(base['G1'])
base['G2'] = label_encoder_g2.fit_transform(base['G2'])
base['absences'] = label_encoder_absences.fit_transform(base['absences'])

In [8]:
base

,absences,failures,higher,G1,G2,G3
0,1,0,1,2,3,Sufficient
1,1,0,1,5,3,Sufficient
2,2,0,1,3,3,Sufficient
3,1,0,1,1,1,Good
4,1,0,1,3,3,Sufficient
...,...,...,...,...,...,...
644,1,1,1,3,3,Sufficient
645,1,0,1,1,1,Very good
646,2,0,1,3,3,Weak
647,2,0,1,3,3,Sufficient


In [9]:
# Separe seus recursos (X) e rótulos (y)
y = base['G3']
X = base.drop('G3', axis=1)

In [10]:
# Crie uma instância do RandomOverSampler
oversampler = RandomOverSampler(random_state=42)
X_resampled, y_resampled = oversampler.fit_resample(X, y)

In [11]:
# Use value_counts() para contar os valores únicos na série
contagem_classes = y_resampled.value_counts()

# A contagem das classes será armazenada em 'contagem_classes'
# Você pode acessar o número de 0s e 1s da seguinte forma:
numero_de_0s = contagem_classes.get(0, 0)  # O segundo argumento é o valor padrão se 0 não estiver presente
numero_de_1s = contagem_classes.get(1, 0)  # O segundo argumento é o valor padrão se 1 não estiver presente
numero_de_2s = contagem_classes.get(2, 0)  # O segundo argumento é o valor padrão se 1 não estiver presente
numero_de_3s = contagem_classes.get(3, 0)  # O segundo argumento é o valor padrão se 1 não estiver presente
numero_de_4s = contagem_classes.get(4, 0)  # O segundo argumento é o valor padrão se 1 não estiver presente
numero_de_5s = contagem_classes.get(5, 0)  # O segundo argumento é o valor padrão se 1 não estiver presente

# Imprima os resultados
print(f"Número de 0s na série: {numero_de_0s}")
print(f"Número de 1s na série: {numero_de_1s}")
print(f"Número de 2s na série: {numero_de_2s}")
print(f"Número de 3s na série: {numero_de_3s}")
print(f"Número de 4s na série: {numero_de_4s}")
print(f"Número de 5s na série: {numero_de_5s}")

Número de 0s na série: 355
Número de 1s na série: 355
Número de 2s na série: 355
Número de 3s na série: 355
Número de 4s na série: 355
Número de 5s na série: 355


In [12]:
label_encoder_teste = LabelEncoder()
y_encoded = label_encoder_teste.fit_transform(y_resampled)

In [13]:
#Inicializar naive bayes
gnb = GaussianNB()
gnb.fit(X_resampled,y_encoded)

GaussianNB()

Agora,vetores de teste para achar as possibilidades de cada combinação de entradas

In [19]:
vetor_teste_low_0_higherY = [[1,0,0,1,1],
               [1,0,0,1,2],
               [1,0,0,1,3],
               [1,0,0,1,4],
               [1,0,0,1,5],
               [1,0,0,2,1],
               [1,0,0,2,2],
               [1,0,0,2,3],
               [1,0,0,2,4],
               [1,0,0,2,5],
               [1,0,0,3,1],
               [1,0,0,3,2],
               [1,0,0,3,3],
               [1,0,0,3,4],
               [1,0,0,3,5],
               [1,0,0,4,1],
               [1,0,0,4,2],
               [1,0,0,4,3],
               [1,0,0,4,4],
               [1,0,0,4,5],
               [1,0,0,5,1],
               [1,0,0,5,2],
               [1,0,0,5,3],
               [1,0,0,5,4],
               [1,0,0,5,5],]

gnb.predict_proba(vetor_teste_low_0_higherY)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


array([[0.00000000e+00, 3.36009770e-09, 9.68292410e-01, 3.17075863e-02,
        0.00000000e+00, 6.24651883e-14],
       [0.00000000e+00, 1.44167452e-09, 7.76612662e-01, 2.23387337e-01,
        0.00000000e+00, 3.45486022e-11],
       [0.00000000e+00, 3.53279112e-10, 5.83261752e-01, 4.16738245e-01,
        0.00000000e+00, 2.87522513e-09],
       [0.00000000e+00, 8.02577919e-11, 6.65835141e-01, 3.34164800e-01,
        0.00000000e+00, 5.84447183e-08],
       [0.00000000e+00, 1.33042945e-11, 9.09349844e-01, 9.06499277e-02,
        0.00000000e+00, 2.28385017e-07],
       [0.00000000e+00, 1.07545892e-11, 9.99577406e-01, 4.22594460e-04,
        0.00000000e+00, 1.66153403e-12],
       [0.00000000e+00, 5.73435966e-12, 9.96300057e-01, 3.69994230e-03,
        0.00000000e+00, 1.14202985e-09],
       [0.00000000e+00, 1.86079517e-12, 9.90859528e-01, 9.14034659e-03,
        0.00000000e+00, 1.25858263e-07],
       [0.00000000e+00, 3.71318495e-13, 9.93559934e-01, 6.43781914e-03,
        0.00000000e+00, 

In [20]:
vetor_teste_low_0_higherN = [[1,0,1,1,1],
               [1,0,1,1,2],
               [1,0,1,1,3],
               [1,0,1,1,4],
               [1,0,1,1,5],
               [1,0,1,2,1],
               [1,0,1,2,2],
               [1,0,1,2,3],
               [1,0,1,2,4],
               [1,0,1,2,5],
               [1,0,1,3,1],
               [1,0,1,3,2],
               [1,0,1,3,3],
               [1,0,1,3,4],
               [1,0,1,3,5],
               [1,0,1,4,1],
               [1,0,1,4,2],
               [1,0,1,4,3],
               [1,0,1,4,4],
               [1,0,1,4,5],
               [1,0,1,5,1],
               [1,0,1,5,2],
               [1,0,1,5,3],
               [1,0,1,5,4],
               [1,0,1,5,5],]

gnb.predict_proba(vetor_teste_low_0_higherN)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


array([[9.99612484e-01, 2.27020842e-07, 3.75730069e-11, 4.62888964e-11,
        3.87288670e-04, 2.02305706e-24],
       [9.99429947e-01, 2.92132485e-07, 9.03800746e-11, 9.78072385e-10,
        5.69759482e-04, 3.35582545e-21],
       [9.99275423e-01, 1.83752131e-07, 1.74234246e-10, 4.68357630e-09,
        7.24387927e-04, 7.16873509e-19],
       [9.99203966e-01, 5.64998452e-08, 2.69204530e-10, 5.08300538e-09,
        7.95971751e-04, 1.97224823e-17],
       [9.99244057e-01, 8.49252556e-09, 3.33373910e-10, 1.25029442e-09,
        7.55932993e-04, 6.98825711e-17],
       [9.99535955e-01, 3.13009948e-07, 1.67084570e-08, 2.65758970e-10,
        4.63715375e-04, 2.31808742e-20],
       [9.99317381e-01, 4.02769518e-07, 4.01899459e-08, 5.61521560e-09,
        6.82169971e-04, 3.84507994e-17],
       [9.99132363e-01, 2.53335390e-07, 7.74756064e-08, 2.68880791e-08,
        8.67279295e-04, 8.21363274e-15],
       [9.99046803e-01, 7.78941071e-08, 1.19703732e-07, 2.91807608e-08,
        9.52970143e-04, 

In [21]:
vetor_teste_low_1_higherY = [[1,1,0,1,1],
               [1,1,0,1,2],
               [1,1,0,1,3],
               [1,1,0,1,4],
               [1,1,0,1,5],
               [1,1,0,2,1],
               [1,1,0,2,2],
               [1,1,0,2,3],
               [1,1,0,2,4],
               [1,1,0,2,5],
               [1,1,0,3,1],
               [1,1,0,3,2],
               [1,1,0,3,3],
               [1,1,0,3,4],
               [1,1,0,3,5],
               [1,1,0,4,1],
               [1,1,0,4,2],
               [1,1,0,4,3],
               [1,1,0,4,4],
               [1,1,0,4,5],
               [1,1,0,5,1],
               [1,1,0,5,2],
               [1,1,0,5,3],
               [1,1,0,5,4],
               [1,1,0,5,5],]

gnb.predict_proba(vetor_teste_low_1_higherY)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


array([[0.00000000e+00, 6.85691716e-19, 9.93868360e-01, 6.13163982e-03,
        0.00000000e+00, 4.75519898e-14],
       [0.00000000e+00, 3.50104091e-19, 9.48592680e-01, 5.14073205e-02,
        0.00000000e+00, 3.12978199e-11],
       [0.00000000e+00, 1.06135555e-19, 8.81356825e-01, 1.18643172e-01,
        0.00000000e+00, 3.22231827e-09],
       [0.00000000e+00, 2.18946238e-20, 9.13613131e-01, 8.63868098e-02,
        0.00000000e+00, 5.94770190e-08],
       [0.00000000e+00, 2.85518618e-21, 9.81564675e-01, 1.84351426e-02,
        0.00000000e+00, 1.82836895e-07],
       [0.00000000e+00, 2.13893439e-21, 9.99920354e-01, 7.96459860e-05,
        0.00000000e+00, 1.23272591e-12],
       [0.00000000e+00, 1.14352503e-21, 9.99300814e-01, 6.99185082e-04,
        0.00000000e+00, 8.49555672e-10],
       [0.00000000e+00, 3.72723661e-22, 9.98264954e-01, 1.73495220e-03,
        0.00000000e+00, 9.40424145e-08],
       [0.00000000e+00, 7.42124195e-23, 9.98779040e-01, 1.21928502e-03,
        0.00000000e+00, 

In [22]:
vetor_teste_low_1_higherN = [[1,1,1,1,1],
               [1,1,1,1,2],
               [1,1,1,1,3],
               [1,1,1,1,4],
               [1,1,1,1,5],
               [1,1,1,2,1],
               [1,1,1,2,2],
               [1,1,1,2,3],
               [1,1,1,2,4],
               [1,1,1,2,5],
               [1,1,1,3,1],
               [1,1,1,3,2],
               [1,1,1,3,3],
               [1,1,1,3,4],
               [1,1,1,3,5],
               [1,1,1,4,1],
               [1,1,1,4,2],
               [1,1,1,4,3],
               [1,1,1,4,4],
               [1,1,1,4,5],
               [1,1,1,5,1],
               [1,1,1,5,2],
               [1,1,1,5,3],
               [1,1,1,5,4],
               [1,1,1,5,5],]

gnb.predict_proba(vetor_teste_low_1_higherN)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


array([[0.00000000e+00, 9.74978150e-07, 8.11615710e-01, 1.88383315e-01,
        1.64565234e-19, 3.24108879e-14],
       [0.00000000e+00, 2.11470447e-07, 3.29069678e-01, 6.70930111e-01,
        4.08070817e-20, 9.06197454e-12],
       [0.00000000e+00, 3.45747987e-08, 1.64894488e-01, 8.35105477e-01,
        1.34856731e-20, 5.03180095e-10],
       [0.00000000e+00, 9.15598421e-09, 2.19424683e-01, 7.80575296e-01,
        1.27623185e-20, 1.19226555e-08],
       [0.00000000e+00, 2.96776078e-09, 5.85961359e-01, 4.14038547e-01,
        2.61366231e-20, 9.10991966e-08],
       [0.00000000e+00, 3.71344461e-09, 9.97012254e-01, 2.98774273e-03,
        5.44307669e-22, 1.02589347e-12],
       [0.00000000e+00, 1.94137597e-09, 9.74351853e-01, 2.56481442e-02,
        3.25326768e-22, 6.91372454e-10],
       [0.00000000e+00, 6.10208780e-10, 9.38626718e-01, 6.13732074e-02,
        2.06688214e-22, 7.38025746e-08],
       [0.00000000e+00, 1.23694130e-10, 9.56087234e-01, 4.39114274e-02,
        1.49726243e-22, 

In [23]:
vetor_teste_low_2_higherY = [[1,2,0,1,1],
               [1,2,0,1,2],
               [1,2,0,1,3],
               [1,2,0,1,4],
               [1,2,0,1,5],
               [1,2,0,2,1],
               [1,2,0,2,2],
               [1,2,0,2,3],
               [1,2,0,2,4],
               [1,2,0,2,5],
               [1,2,0,3,1],
               [1,2,0,3,2],
               [1,2,0,3,3],
               [1,2,0,3,4],
               [1,2,0,3,5],
               [1,2,0,4,1],
               [1,2,0,4,2],
               [1,2,0,4,3],
               [1,2,0,4,4],
               [1,2,0,4,5],
               [1,2,0,5,1],
               [1,2,0,5,2],
               [1,2,0,5,3],
               [1,2,0,5,4],
               [1,2,0,5,5],]

gnb.predict_proba(vetor_teste_low_2_higherY)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


array([[0.00000000e+00, 2.60410616e-47, 9.99859659e-01, 1.40340848e-04,
        0.00000000e+00, 5.66356883e-14],
       [0.00000000e+00, 1.39155984e-47, 9.98768577e-01, 1.23142325e-03,
        0.00000000e+00, 3.90131031e-11],
       [0.00000000e+00, 4.53211259e-48, 9.96946761e-01, 3.05323494e-03,
        0.00000000e+00, 4.31519013e-09],
       [0.00000000e+00, 9.02737468e-49, 9.97853333e-01, 2.14658997e-03,
        0.00000000e+00, 7.69068247e-08],
       [0.00000000e+00, 1.09761362e-49, 9.99572670e-01, 4.27109503e-04,
        0.00000000e+00, 2.20429947e-07],
       [0.00000000e+00, 8.07515503e-50, 9.99998188e-01, 1.81215348e-06,
        0.00000000e+00, 1.45952532e-12],
       [0.00000000e+00, 4.31978489e-50, 9.99984081e-01, 1.59179186e-05,
        0.00000000e+00, 1.00646806e-09],
       [0.00000000e+00, 1.40943003e-50, 9.99960350e-01, 3.95386421e-05,
        0.00000000e+00, 1.11524949e-07],
       [0.00000000e+00, 2.80487718e-51, 9.99970241e-01, 2.77728324e-05,
        0.00000000e+00, 

In [24]:
vetor_teste_low_2_higherN = [[1,2,1,1,1],
               [1,2,1,1,2],
               [1,2,1,1,3],
               [1,2,1,1,4],
               [1,2,1,1,5],
               [1,2,1,2,1],
               [1,2,1,2,2],
               [1,2,1,2,3],
               [1,2,1,2,4],
               [1,2,1,2,5],
               [1,2,1,3,1],
               [1,2,1,3,2],
               [1,2,1,3,3],
               [1,2,1,3,4],
               [1,2,1,3,5],
               [1,2,1,4,1],
               [1,2,1,4,2],
               [1,2,1,4,3],
               [1,2,1,4,4],
               [1,2,1,4,5],
               [1,2,1,5,1],
               [1,2,1,5,2],
               [1,2,1,5,3],
               [1,2,1,5,4],
               [1,2,1,5,5],]

gnb.predict_proba(vetor_teste_low_2_higherN)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


array([[0.00000000e+000, 4.51104029e-035, 9.94747066e-001,
        5.25293390e-003, 2.68229140e-096, 4.70288606e-014],
       [0.00000000e+000, 2.31840608e-035, 9.55670301e-001,
        4.43296994e-002, 1.57601885e-096, 3.11569092e-011],
       [0.00000000e+000, 7.09760944e-036, 8.96683077e-001,
        1.03316919e-001, 9.75239106e-097, 3.23942251e-009],
       [0.00000000e+000, 1.45727087e-036, 9.25126511e-001,
        7.48734299e-002, 7.15567931e-097, 5.95113700e-008],
       [0.00000000e+000, 1.88171299e-037, 9.84178490e-001,
        1.58213285e-002, 5.83794517e-097, 1.81146833e-007],
       [0.00000000e+000, 1.40593890e-037, 9.99931827e-001,
        6.81726812e-005, 7.25972206e-099, 1.21810147e-012],
       [0.00000000e+000, 7.51715359e-038, 9.99401481e-001,
        5.98518252e-004, 4.43761496e-099, 8.39551907e-010],
       [0.00000000e+000, 2.45052721e-038, 9.98514527e-001,
        1.48538001e-003, 2.92403679e-099, 9.29489087e-008],
       [0.00000000e+000, 4.87884357e-039, 9.9895

In [25]:
vetor_teste_high_0_higherY = [[0,0,0,1,1],
               [0,0,0,1,2],
               [0,0,0,1,3],
               [0,0,0,1,4],
               [0,0,0,1,5],
               [0,0,0,2,1],
               [0,0,0,2,2],
               [0,0,0,2,3],
               [0,0,0,2,4],
               [0,0,0,2,5],
               [0,0,0,3,1],
               [0,0,0,3,2],
               [0,0,0,3,3],
               [0,0,0,3,4],
               [0,0,0,3,5],
               [0,0,0,4,1],
               [0,0,0,4,2],
               [0,0,0,4,3],
               [0,0,0,4,4],
               [0,0,0,4,5],
               [0,0,0,5,1],
               [0,0,0,5,2],
               [0,0,0,5,3],
               [0,0,0,5,4],
               [0,0,0,5,5],]

gnb.predict_proba(vetor_teste_high_0_higherY)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


array([[0.00000000e+00, 6.15093029e-08, 0.00000000e+00, 9.99999938e-01,
        0.00000000e+00, 3.83362330e-12],
       [0.00000000e+00, 3.74593969e-09, 0.00000000e+00, 9.99999996e-01,
        0.00000000e+00, 3.00958436e-10],
       [0.00000000e+00, 4.92047132e-10, 0.00000000e+00, 9.99999986e-01,
        0.00000000e+00, 1.34258899e-08],
       [0.00000000e+00, 1.39405038e-10, 0.00000000e+00, 9.99999660e-01,
        0.00000000e+00, 3.40344716e-07],
       [0.00000000e+00, 8.51872047e-11, 0.00000000e+00, 9.99995097e-01,
        0.00000000e+00, 4.90266941e-06],
       [0.00000000e+00, 1.47714159e-08, 0.00000000e+00, 9.99999978e-01,
        0.00000000e+00, 7.65103210e-09],
       [0.00000000e+00, 8.99584212e-10, 0.00000000e+00, 9.99999398e-01,
        0.00000000e+00, 6.00643571e-07],
       [0.00000000e+00, 1.18161604e-10, 0.00000000e+00, 9.99973206e-01,
        0.00000000e+00, 2.67942758e-05],
       [0.00000000e+00, 3.34553074e-11, 0.00000000e+00, 9.99321211e-01,
        0.00000000e+00, 

In [26]:
vetor_teste_high_0_higherN = [[0,0,1,1,1],
               [0,0,1,1,2],
               [0,0,1,1,3],
               [0,0,1,1,4],
               [0,0,1,1,5],
               [0,0,1,2,1],
               [0,0,1,2,2],
               [0,0,1,2,3],
               [0,0,1,2,4],
               [0,0,1,2,5],
               [0,0,1,3,1],
               [0,0,1,3,2],
               [0,0,1,3,3],
               [0,0,1,3,4],
               [0,0,1,3,5],
               [0,0,1,4,1],
               [0,0,1,4,2],
               [0,0,1,4,3],
               [0,0,1,4,4],
               [0,0,1,4,5],
               [0,0,1,5,1],
               [0,0,1,5,2],
               [0,0,1,5,3],
               [0,0,1,5,4],
               [0,0,1,5,5],]

gnb.predict_proba(vetor_teste_high_0_higherN)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


array([[9.99262853e-01, 5.28580485e-07, 0.00000000e+00, 1.85682131e-10,
        7.36618713e-04, 1.57919590e-23],
       [9.98915818e-01, 6.80070141e-07, 0.00000000e+00, 3.92276907e-09,
        1.08349766e-03, 2.61912183e-20],
       [9.98622195e-01, 4.27706366e-07, 0.00000000e+00, 1.87818696e-08,
        1.37735882e-03, 5.59420459e-18],
       [9.98486477e-01, 1.31502079e-07, 0.00000000e+00, 2.03823296e-08,
        1.51337159e-03, 1.53896739e-16],
       [9.98562677e-01, 1.97668716e-08, 0.00000000e+00, 5.01373368e-09,
        1.43729839e-03, 5.45321268e-16],
       [9.99117350e-01, 7.28741568e-07, 0.00000000e+00, 1.06598532e-09,
        8.81920544e-04, 1.80937129e-19],
       [9.98701906e-01, 9.37532552e-07, 0.00000000e+00, 2.25187400e-08,
        1.29713423e-03, 3.00066572e-16],
       [9.98350461e-01, 5.89594173e-07, 0.00000000e+00, 1.07811475e-07,
        1.64884130e-03, 6.40877585e-14],
       [9.98188087e-01, 1.81271060e-07, 0.00000000e+00, 1.16995289e-07,
        1.81161424e-03, 

In [27]:
vetor_teste_high_1_higherY = [[0,1,0,1,1],
               [0,1,0,1,2],
               [0,1,0,1,3],
               [0,1,0,1,4],
               [0,1,0,1,5],
               [0,1,0,2,1],
               [0,1,0,2,2],
               [0,1,0,2,3],
               [0,1,0,2,4],
               [0,1,0,2,5],
               [0,1,0,3,1],
               [0,1,0,3,2],
               [0,1,0,3,3],
               [0,1,0,3,4],
               [0,1,0,3,5],
               [0,1,0,4,1],
               [0,1,0,4,2],
               [0,1,0,4,3],
               [0,1,0,4,4],
               [0,1,0,4,5],
               [0,1,0,5,1],
               [0,1,0,5,2],
               [0,1,0,5,3],
               [0,1,0,5,4],
               [0,1,0,5,5],]

gnb.predict_proba(vetor_teste_high_1_higherY)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


array([[0.00000000e+00, 6.49089189e-17, 0.00000000e+00, 1.00000000e+00,
        0.00000000e+00, 1.50912999e-11],
       [0.00000000e+00, 3.95297736e-18, 0.00000000e+00, 9.99999999e-01,
        0.00000000e+00, 1.18474180e-09],
       [0.00000000e+00, 5.19242500e-19, 0.00000000e+00, 9.99999947e-01,
        0.00000000e+00, 5.28518573e-08],
       [0.00000000e+00, 1.47109791e-19, 0.00000000e+00, 9.99998660e-01,
        0.00000000e+00, 1.33978702e-06],
       [0.00000000e+00, 8.98941975e-20, 0.00000000e+00, 9.99980701e-01,
        0.00000000e+00, 1.92993884e-05],
       [0.00000000e+00, 1.55878302e-17, 0.00000000e+00, 9.99999970e-01,
        0.00000000e+00, 3.01187684e-08],
       [0.00000000e+00, 9.49302454e-19, 0.00000000e+00, 9.99997636e-01,
        0.00000000e+00, 2.36446699e-06],
       [0.00000000e+00, 1.24682572e-19, 0.00000000e+00, 9.99894531e-01,
        0.00000000e+00, 1.05469052e-04],
       [0.00000000e+00, 3.52341459e-20, 0.00000000e+00, 9.97333220e-01,
        0.00000000e+00, 

In [ ]:
vetor_teste_high_1_higherN = [[0,1,1,1,1],
               [0,1,1,1,2],
               [0,1,1,1,3],
               [0,1,1,1,4],
               [0,1,1,1,5],
               [0,1,1,2,1],
               [0,1,1,2,2],
               [0,1,1,2,3],
               [0,1,1,2,4],
               [0,1,1,2,5],
               [0,1,1,3,1],
               [0,1,1,3,2],
               [0,1,1,3,3],
               [0,1,1,3,4],
               [0,1,1,3,5],
               [0,1,1,4,1],
               [0,1,1,4,2],
               [0,1,1,4,3],
               [0,1,1,4,4],
               [0,1,1,4,5],
               [0,1,1,5,1],
               [0,1,1,5,2],
               [0,1,1,5,3],
               [0,1,1,5,4],
               [0,1,1,5,5],]

gnb.predict_proba(vetor_teste_high_1_higherN)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


array([[0.00000000e+00, 9.90879061e-14, 0.00000000e+00, 1.00000000e+00,
        2.60197324e-28, 1.23262948e-13],
       [0.00000000e+00, 6.97008015e-15, 0.00000000e+00, 1.00000000e+00,
        2.02290943e-29, 9.13086696e-12],
       [0.00000000e+00, 9.73625610e-16, 0.00000000e+00, 1.00000000e+00,
        5.52190571e-30, 3.71799465e-10],
       [0.00000000e+00, 2.70074324e-16, 0.00000000e+00, 9.99999992e-01,
        5.29225139e-30, 8.32190555e-09],
       [0.00000000e+00, 1.48768733e-16, 0.00000000e+00, 9.99999898e-01,
        1.78086436e-29, 1.02389176e-07],
       [0.00000000e+00, 2.40264001e-14, 0.00000000e+00, 1.00000000e+00,
        5.33730200e-29, 4.38770978e-10],
       [0.00000000e+00, 1.69007436e-15, 0.00000000e+00, 9.99999967e-01,
        4.14949625e-30, 3.25025434e-08],
       [0.00000000e+00, 2.36080148e-16, 0.00000000e+00, 9.99998677e-01,
        1.13268034e-30, 1.32346826e-06],
       [0.00000000e+00, 6.54844962e-17, 0.00000000e+00, 9.99970378e-01,
        1.08554181e-30, 

In [ ]:
vetor_teste_high_2_higherY = [[0,2,0,1,1],
               [0,2,0,1,2],
               [0,2,0,1,3],
               [0,2,0,1,4],
               [0,2,0,1,5],
               [0,2,0,2,1],
               [0,2,0,2,2],
               [0,2,0,2,3],
               [0,2,0,2,4],
               [0,2,0,2,5],
               [0,2,0,3,1],
               [0,2,0,3,2],
               [0,2,0,3,3],
               [0,2,0,3,4],
               [0,2,0,3,5],
               [0,2,0,4,1],
               [0,2,0,4,2],
               [0,2,0,4,3],
               [0,2,0,4,4],
               [0,2,0,4,5],
               [0,2,0,5,1],
               [0,2,0,5,2],
               [0,2,0,5,3],
               [0,2,0,5,4],
               [0,2,0,5,5],]

gnb.predict_proba(vetor_teste_high_2_higherY)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


array([[0.00000000e+00, 1.49901126e-72, 0.00000000e+00, 1.00000000e+00,
        0.00000000e+00, 1.69816309e-10],
       [0.00000000e+00, 1.05444033e-73, 0.00000000e+00, 9.99999987e-01,
        0.00000000e+00, 1.25793690e-08],
       [0.00000000e+00, 1.47290932e-74, 0.00000000e+00, 9.99999488e-01,
        0.00000000e+00, 5.12218649e-07],
       [0.00000000e+00, 4.08566323e-75, 0.00000000e+00, 9.99988535e-01,
        0.00000000e+00, 1.14647517e-05],
       [0.00000000e+00, 2.25027034e-75, 0.00000000e+00, 9.99858961e-01,
        0.00000000e+00, 1.41039150e-04],
       [0.00000000e+00, 3.63473444e-73, 0.00000000e+00, 9.99999396e-01,
        0.00000000e+00, 6.04483540e-07],
       [0.00000000e+00, 2.55664615e-74, 0.00000000e+00, 9.99955224e-01,
        0.00000000e+00, 4.47759452e-05],
       [0.00000000e+00, 3.56494762e-75, 0.00000000e+00, 9.98180007e-01,
        0.00000000e+00, 1.81999343e-03],
       [0.00000000e+00, 9.51839698e-76, 0.00000000e+00, 9.60789426e-01,
        0.00000000e+00, 

In [ ]:
vetor_teste_high_2_higherN = [[0,2,1,1,1],
               [0,2,1,1,2],
               [0,2,1,1,3],
               [0,2,1,1,4],
               [0,2,1,1,5],
               [0,2,1,2,1],
               [0,2,1,2,2],
               [0,2,1,2,3],
               [0,2,1,2,4],
               [0,2,1,2,5],
               [0,2,1,3,1],
               [0,2,1,3,2],
               [0,2,1,3,3],
               [0,2,1,3,4],
               [0,2,1,3,5],
               [0,2,1,4,1],
               [0,2,1,4,2],
               [0,2,1,4,3],
               [0,2,1,4,4],
               [0,2,1,4,5],
               [0,2,1,5,1],
               [0,2,1,5,2],
               [0,2,1,5,3],
               [0,2,1,5,4],
               [0,2,1,5,5],]

gnb.predict_proba(vetor_teste_high_2_higherN)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


array([[0.00000000e+000, 3.47260443e-063, 0.00000000e+000,
        1.00000000e+000, 1.85690593e-131, 5.01701256e-012],
       [0.00000000e+000, 2.44271296e-064, 0.00000000e+000,
        1.00000000e+000, 1.44365532e-132, 3.71641885e-010],
       [0.00000000e+000, 3.41213847e-065, 0.00000000e+000,
        9.99999985e-001, 3.94072435e-133, 1.51328732e-008],
       [0.00000000e+000, 9.46493899e-066, 0.00000000e+000,
        9.99999661e-001, 3.77682967e-133, 3.38715662e-007],
       [0.00000000e+000, 5.21368230e-066, 0.00000000e+000,
        9.99995833e-001, 1.27091397e-132, 4.16739744e-006],
       [0.00000000e+000, 8.42021850e-064, 0.00000000e+000,
        9.99999982e-001, 3.80898136e-132, 1.78587281e-008],
       [0.00000000e+000, 5.92297523e-065, 0.00000000e+000,
        9.99998677e-001, 2.96129650e-133, 1.32290734e-006],
       [0.00000000e+000, 8.27315791e-066, 0.00000000e+000,
        9.99946135e-001, 8.08298220e-134, 5.38645892e-005],
       [0.00000000e+000, 2.29225382e-066, 0.0000

In [ ]:
vetor_teste_medium_0_higherY = [[2,0,0,1,1],
               [2,0,0,1,2],
               [2,0,0,1,3],
               [2,0,0,1,4],
               [2,0,0,1,5],
               [2,0,0,2,1],
               [2,0,0,2,2],
               [2,0,0,2,3],
               [2,0,0,2,4],
               [2,0,0,2,5],
               [2,0,0,3,1],
               [2,0,0,3,2],
               [2,0,0,3,3],
               [2,0,0,3,4],
               [2,0,0,3,5],
               [2,0,0,4,1],
               [2,0,0,4,2],
               [2,0,0,4,3],
               [2,0,0,4,4],
               [2,0,0,4,5],
               [2,0,0,5,1],
               [2,0,0,5,2],
               [2,0,0,5,3],
               [2,0,0,5,4],
               [2,0,0,5,5],]

gnb.predict_proba(vetor_teste_medium_0_higherY)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


array([[0.00000000e+00, 1.58691498e-06, 0.00000000e+00, 9.99998413e-01,
        0.00000000e+00, 8.53405745e-13],
       [0.00000000e+00, 1.11627557e-07, 0.00000000e+00, 9.99999888e-01,
        0.00000000e+00, 6.32172601e-11],
       [0.00000000e+00, 1.55928564e-08, 0.00000000e+00, 9.99999982e-01,
        0.00000000e+00, 2.57414173e-09],
       [0.00000000e+00, 4.32530734e-09, 0.00000000e+00, 9.99999938e-01,
        0.00000000e+00, 5.76164455e-08],
       [0.00000000e+00, 2.38256686e-09, 0.00000000e+00, 9.99999289e-01,
        0.00000000e+00, 7.08887771e-07],
       [0.00000000e+00, 3.84788633e-07, 0.00000000e+00, 9.99999612e-01,
        0.00000000e+00, 3.03781574e-09],
       [0.00000000e+00, 2.70669558e-08, 0.00000000e+00, 9.99999748e-01,
        0.00000000e+00, 2.25030276e-07],
       [0.00000000e+00, 3.78085272e-09, 0.00000000e+00, 9.99990833e-01,
        0.00000000e+00, 9.16291859e-06],
       [0.00000000e+00, 1.04856653e-09, 0.00000000e+00, 9.99794948e-01,
        0.00000000e+00, 

In [ ]:
vetor_teste_medium_0_higherN = [[2,0,1,1,1],
               [2,0,1,1,2],
               [2,0,1,1,3],
               [2,0,1,1,4],
               [2,0,1,1,5],
               [2,0,1,2,1],
               [2,0,1,2,2],
               [2,0,1,2,3],
               [2,0,1,2,4],
               [2,0,1,2,5],
               [2,0,1,3,1],
               [2,0,1,3,2],
               [2,0,1,3,3],
               [2,0,1,3,4],
               [2,0,1,3,5],
               [2,0,1,4,1],
               [2,0,1,4,2],
               [2,0,1,4,3],
               [2,0,1,4,4],
               [2,0,1,4,5],
               [2,0,1,5,1],
               [2,0,1,5,2],
               [2,0,1,5,3],
               [2,0,1,5,4],
               [2,0,1,5,5],]

gnb.predict_proba(vetor_teste_medium_0_higherN)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


array([[9.81879284e-01, 1.08738710e-05, 0.00000000e+00, 2.95787222e-09,
        1.81098394e-02, 7.45764064e-23],
       [9.73484222e-01, 1.43972877e-05, 0.00000000e+00, 5.56748162e-08,
        2.65013254e-02, 1.03982511e-19],
       [9.66241058e-01, 9.38252041e-06, 0.00000000e+00, 2.59742736e-07,
        3.37492993e-02, 1.97533912e-17],
       [9.62501148e-01, 3.01699180e-06, 0.00000000e+00, 3.01097195e-07,
        3.74955340e-02, 5.12529645e-16],
       [9.63605016e-01, 4.79366135e-07, 0.00000000e+00, 8.68503398e-08,
        3.63944175e-02, 1.81892632e-15],
       [9.78726049e-01, 1.50889269e-05, 0.00000000e+00, 1.69272158e-08,
        2.12588454e-02, 1.51919289e-18],
       [9.68916473e-01, 1.99484535e-05, 0.00000000e+00, 3.18140740e-07,
        3.10632607e-02, 2.11507680e-15],
       [9.60477231e-01, 1.29835134e-05, 0.00000000e+00, 1.48234082e-06,
        3.95083026e-02, 4.01283839e-13],
       [9.56129234e-01, 4.17215657e-06, 0.00000000e+00, 1.71721678e-06,
        4.38648763e-02, 

In [ ]:
vetor_teste_medium_1_higherY = [[2,1,0,1,1],
               [2,1,0,1,2],
               [2,1,0,1,3],
               [2,1,0,1,4],
               [2,1,0,1,5],
               [2,1,0,2,1],
               [2,1,0,2,2],
               [2,1,0,2,3],
               [2,1,0,2,4],
               [2,1,0,2,5],
               [2,1,0,3,1],
               [2,1,0,3,2],
               [2,1,0,3,3],
               [2,1,0,3,4],
               [2,1,0,3,5],
               [2,1,0,4,1],
               [2,1,0,4,2],
               [2,1,0,4,3],
               [2,1,0,4,4],
               [2,1,0,4,5],
               [2,1,0,5,1],
               [2,1,0,5,2],
               [2,1,0,5,3],
               [2,1,0,5,4],
               [2,1,0,5,5],]

gnb.predict_proba(vetor_teste_medium_1_higherY)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


array([[0.00000000e+00, 4.71212861e-23, 0.00000000e+00, 1.00000000e+00,
        0.00000000e+00, 3.13366297e-12],
       [0.00000000e+00, 3.31462390e-24, 0.00000000e+00, 1.00000000e+00,
        0.00000000e+00, 2.32130257e-10],
       [0.00000000e+00, 4.63007973e-25, 0.00000000e+00, 9.99999991e-01,
        0.00000000e+00, 9.45210404e-09],
       [0.00000000e+00, 1.28433907e-25, 0.00000000e+00, 9.99999788e-01,
        0.00000000e+00, 2.11564322e-07],
       [0.00000000e+00, 7.07468428e-26, 0.00000000e+00, 9.99997397e-01,
        0.00000000e+00, 2.60299118e-06],
       [0.00000000e+00, 1.14257623e-23, 0.00000000e+00, 9.99999989e-01,
        0.00000000e+00, 1.11546931e-08],
       [0.00000000e+00, 8.03714775e-25, 0.00000000e+00, 9.99999174e-01,
        0.00000000e+00, 8.26298069e-07],
       [0.00000000e+00, 1.12264363e-25, 0.00000000e+00, 9.99966355e-01,
        0.00000000e+00, 3.36448991e-05],
       [0.00000000e+00, 3.11186609e-26, 0.00000000e+00, 9.99247475e-01,
        0.00000000e+00, 

In [ ]:
vetor_teste_medium_1_higherN = [[2,1,1,1,1],
               [2,1,1,1,2],
               [2,1,1,1,3],
               [2,1,1,1,4],
               [2,1,1,1,5],
               [2,1,1,2,1],
               [2,1,1,2,2],
               [2,1,1,2,3],
               [2,1,1,2,4],
               [2,1,1,2,5],
               [2,1,1,3,1],
               [2,1,1,3,2],
               [2,1,1,3,3],
               [2,1,1,3,4],
               [2,1,1,3,5],
               [2,1,1,4,1],
               [2,1,1,4,2],
               [2,1,1,4,3],
               [2,1,1,4,4],
               [2,1,1,4,5],
               [2,1,1,5,1],
               [2,1,1,5,2],
               [2,1,1,5,3],
               [2,1,1,5,4],
               [2,1,1,5,5],]

gnb.predict_proba(vetor_teste_medium_1_higherN)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


array([[0.00000000e+00, 1.09161013e-13, 0.00000000e+00, 1.00000000e+00,
        3.22599355e-28, 9.25801919e-14],
       [0.00000000e+00, 7.67864656e-15, 0.00000000e+00, 1.00000000e+00,
        2.50805530e-29, 6.85800099e-12],
       [0.00000000e+00, 1.07260272e-15, 0.00000000e+00, 1.00000000e+00,
        6.84620117e-30, 2.79250712e-10],
       [0.00000000e+00, 2.97529618e-16, 0.00000000e+00, 9.99999994e-01,
        6.56146984e-30, 6.25040720e-09],
       [0.00000000e+00, 1.63892312e-16, 0.00000000e+00, 9.99999923e-01,
        2.20796164e-29, 7.69023455e-08],
       [0.00000000e+00, 2.64688828e-14, 0.00000000e+00, 1.00000000e+00,
        6.61732473e-29, 3.29551598e-10],
       [0.00000000e+00, 1.86188444e-15, 0.00000000e+00, 9.99999976e-01,
        5.14465255e-30, 2.44119728e-08],
       [0.00000000e+00, 2.60079737e-16, 0.00000000e+00, 9.99999006e-01,
        1.40432678e-30, 9.94029339e-07],
       [0.00000000e+00, 7.21420699e-17, 0.00000000e+00, 9.99977751e-01,
        1.34589267e-30, 

In [ ]:
vetor_teste_medium_2_higherY = [[2,2,0,1,1],
               [2,2,0,1,2],
               [2,2,0,1,3],
               [2,2,0,1,4],
               [2,2,0,1,5],
               [2,2,0,2,1],
               [2,2,0,2,2],
               [2,2,0,2,3],
               [2,2,0,2,4],
               [2,2,0,2,5],
               [2,2,0,3,1],
               [2,2,0,3,2],
               [2,2,0,3,3],
               [2,2,0,3,4],
               [2,2,0,3,5],
               [2,2,0,4,1],
               [2,2,0,4,2],
               [2,2,0,4,3],
               [2,2,0,4,4],
               [2,2,0,4,5],
               [2,2,0,5,1],
               [2,2,0,5,2],
               [2,2,0,5,3],
               [2,2,0,5,4],
               [2,2,0,5,5],]

gnb.predict_proba(vetor_teste_medium_2_higherY)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


array([[0.00000000e+00, 1.65139817e-72, 0.00000000e+00, 1.00000000e+00,
        0.00000000e+00, 1.27545437e-10],
       [0.00000000e+00, 1.16163294e-73, 0.00000000e+00, 9.99999991e-01,
        0.00000000e+00, 9.44809795e-09],
       [0.00000000e+00, 1.62264296e-74, 0.00000000e+00, 9.99999615e-01,
        0.00000000e+00, 3.84716639e-07],
       [0.00000000e+00, 4.50101760e-75, 0.00000000e+00, 9.99991389e-01,
        0.00000000e+00, 8.61095704e-06],
       [0.00000000e+00, 2.47911600e-75, 0.00000000e+00, 9.99894065e-01,
        0.00000000e+00, 1.05935241e-04],
       [0.00000000e+00, 4.00423592e-73, 0.00000000e+00, 9.99999546e-01,
        0.00000000e+00, 4.54014865e-07],
       [0.00000000e+00, 2.81658181e-74, 0.00000000e+00, 9.99966369e-01,
        0.00000000e+00, 3.36306399e-05],
       [0.00000000e+00, 3.92913413e-75, 0.00000000e+00, 9.98632422e-01,
        0.00000000e+00, 1.36757812e-03],
       [0.00000000e+00, 1.05893768e-75, 0.00000000e+00, 9.70259490e-01,
        0.00000000e+00, 

In [ ]:
vetor_teste_medium_2_higherN = [[2,2,1,1,1],
               [2,2,1,1,2],
               [2,2,1,1,3],
               [2,2,1,1,4],
               [2,2,1,1,5],
               [2,2,1,2,1],
               [2,2,1,2,2],
               [2,2,1,2,3],
               [2,2,1,2,4],
               [2,2,1,2,5],
               [2,2,1,3,1],
               [2,2,1,3,2],
               [2,2,1,3,3],
               [2,2,1,3,4],
               [2,2,1,3,5],
               [2,2,1,4,1],
               [2,2,1,4,2],
               [2,2,1,4,3],
               [2,2,1,4,4],
               [2,2,1,4,5],
               [2,2,1,5,1],
               [2,2,1,5,2],
               [2,2,1,5,3],
               [2,2,1,5,4],
               [2,2,1,5,5],]

gnb.predict_proba(vetor_teste_medium_2_higherN)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


array([[0.00000000e+000, 3.82562345e-063, 0.00000000e+000,
        1.00000000e+000, 2.30223988e-131, 3.76817199e-012],
       [0.00000000e+000, 2.69103498e-064, 0.00000000e+000,
        1.00000000e+000, 1.78988111e-132, 2.79132357e-010],
       [0.00000000e+000, 3.75901063e-065, 0.00000000e+000,
        9.99999989e-001, 4.88581174e-133, 1.13659810e-008],
       [0.00000000e+000, 1.04271294e-065, 0.00000000e+000,
        9.99999746e-001, 4.68261127e-133, 2.54402189e-007],
       [0.00000000e+000, 5.74370227e-066, 0.00000000e+000,
        9.99996870e-001, 1.57571357e-132, 3.13004729e-006],
       [0.00000000e+000, 9.27620354e-064, 0.00000000e+000,
        9.99999987e-001, 4.72247338e-132, 1.34133129e-008],
       [0.00000000e+000, 6.52509689e-065, 0.00000000e+000,
        9.99999006e-001, 3.67149302e-133, 9.93608039e-007],
       [0.00000000e+000, 9.11431518e-066, 0.00000000e+000,
        9.99959543e-001, 1.00216242e-133, 4.04570958e-005],
       [0.00000000e+000, 2.52603763e-066, 0.0000